## S&P 500 Sector Performance Overview

This notebook loads daily prices for the eleven SPDR sector ETFs (XLC, XLY, XLP, XLE, XLF, XLV, XLI, XLB, XLRE, XLK, XLU), filters the last year of data, and compares their cumulative returns across multiple horizons.

### Data Pipeline
- Download or reuse cached CSVs from `yfinance`, keeping only the `Close` prices per ticker.
- Rename columns so every series shows `Ticker - Sector` in plots and reports.
- Limit the analysis window to the most recent 12 months to highlight current trends.
- Compute simple daily returns with `pct_change()` and roll them into cumulative performance via `(1 + r).cumprod() - 1`.

### Visualization Blocks
1. **Full-year cumulative returns**: compares every sector over the past 12 months.
2. **Last 6 months**: focuses on mid-term rotations and acceleration.
3. **Last 3 months**: captures short-term leadership shifts.

### Key Takeaways
- Identify which sectors lead or lag over each horizon.
- Observe whether leadership is broad-based or concentrated in a few sectors.
- Use the shortened windows to spot emerging trends that may not appear in the full-year chart.

In [5]:
import pandas as pd
import plotly.graph_objects as go

import yfinance as yf

import warnings
warnings.filterwarnings("ignore")

import datetime as dt

In [6]:
# portfolio of sector ETFs and their names
ticker_sector_map = {
    "XLC": "Communication Services",
    "XLY": "Consumer Discretionary",
    "XLP": "Consumer Staples",
    "XLE": "Energy",
    "XLF": "Financials",
    "XLV": "Health Care",
    "XLI": "Industrials",
    "XLB": "Materials",
    "XLRE": "Real Estate",
    "XLK": "Technology",
    "XLU": "Utilities",
}

tickers = list(ticker_sector_map.keys())
sector_labels = {ticker: f"{ticker} - {name}" for ticker, name in ticker_sector_map.items()}

In [7]:
start_date = "2023-01-01"
end_date = dt.datetime.now().strftime("%Y-%m-%d")

# Download historical data from yf API
data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# download data to csv, filename have tickers joined by underscore and end_date
data.to_csv("_".join(tickers) + "_" + end_date + ".csv")

[*********************100%***********************]  11 of 11 completed



In [8]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("_".join(tickers) + "_" + end_date + ".csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Convert all values to float
df = df.astype(float)

# set index as datetime
df.index = pd.to_datetime(df.index)

# keep only level 1 'Close' prices
df = df.xs('Close', level=1, axis=1)

# use descriptive names in charts/legends
df = df.rename(columns=sector_labels)

# Show the result
df.head()

Ticker,XLP - Consumer Staples,XLV - Health Care,XLB - Materials,XLF - Financials,XLE - Energy,XLRE - Real Estate,XLK - Technology,XLY - Consumer Discretionary,XLI - Industrials,XLU - Utilities,XLC - Communication Services
Date,,,,,,,,,,,
2023-01-03,69.052116,129.434219,73.552414,32.818188,76.900307,33.786453,120.799461,125.403618,94.314217,64.770363,47.214478
2023-01-04,69.321739,129.778214,74.831993,33.344124,76.891190,34.561172,121.113060,127.220345,95.147942,65.358337,48.098347
2023-01-05,68.624435,128.450012,73.647194,33.095493,78.294357,33.549492,118.760963,126.038498,94.122566,63.943501,48.166348
2023-01-06,70.456039,129.634872,76.177933,33.831806,79.779495,34.524715,122.240120,128.998032,96.700356,65.229721,48.953091
2023-01-09,69.768036,127.465843,76.661331,33.707493,79.497055,34.506485,123.661163,129.779434,96.269119,65.670715,49.030792


In [9]:
# date filter
# keep only last year of data
one_year_ago = dt.datetime.now() - dt.timedelta(days=365)
df = df[df.index >= one_year_ago]

In [10]:
# df copies for last 6 months and 3 months of data
df_6m = df[df.index >= (dt.datetime.now() - dt.timedelta(days=182))]
df_3m = df[df.index >= (dt.datetime.now() - dt.timedelta(days=91))]

In [11]:
# calculate simple returns with pct_change()
simple_returns = df.pct_change().fillna(0)

# cumulative product of simple returns (correct for compounding)
cumprod_simple = (1 + simple_returns).cumprod() - 1

In [12]:
# plot cumprod_simple
fig = go.Figure()
for column in cumprod_simple.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple.index,
            y=cumprod_simple[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Simple Returns (Last Year)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [13]:
# plot last 6 months only
cumprod_simple_last_6_months = cumprod_simple.last('6M')
fig = go.Figure()
for column in cumprod_simple_last_6_months.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_last_6_months.index,
            y=cumprod_simple_last_6_months[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Simple Returns (Last 6 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [14]:
# plot last 3 months only
cumprod_simple_last_3_months = cumprod_simple.last('3M')
fig = go.Figure()
for column in cumprod_simple_last_3_months.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_last_3_months.index,
            y=cumprod_simple_last_3_months[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Simple Returns (Last 3 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [15]:
# calculate simple returns with pct_change() for 6m and 3m dataframes
simple_returns_6m = df_6m.pct_change().fillna(0)
simple_returns_3m = df_3m.pct_change().fillna(0)

# cumulative product of simple returns (correct for compounding) for 6m and 3m dataframes
cumprod_simple_6m = (1 + simple_returns_6m).cumprod() - 1
cumprod_simple_3m = (1 + simple_returns_3m).cumprod() - 1

In [16]:
# plot last 6 months of cumprod_simple_6m returns 
fig = go.Figure()
for column in cumprod_simple_6m.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_6m.index,
            y=cumprod_simple_6m[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Simple Returns (Last 3 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [17]:
# plot last 3 months of cumprod_simple_3m returns 
fig = go.Figure()
for column in cumprod_simple_3m.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_3m.index,
            y=cumprod_simple_3m[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Simple Returns (Last 3 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()